## NASA L3M2GeoTiff & GEE Upload


## Purpose
Convert Nasa gridded files to geotiff and send to the GEE.

## Methodology
1. Create projection using pyproj to get projected area extent.
2. Create a 2D resampling grid using pyresample AreaDefinition.from_extent. Pyresample needs the following

**Shape** <br>
>Lat: 4320 / number of equal boxes.  <br>
>Lon: 8640 / number of equal boxes.


## Results
Uploaded geotiffs in the GEE


## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [1]:
# Data manipulation
import os
import json
import time
from calendar import timegm
import numpy as np
import pyproj
from pyresample import geometry
from glob import glob

from datetime import datetime
from osgeo import gdal, gdal_array, osr
from netCDF4 import (Dataset, date2num)
from datafetch.nasa_l3 import getfile
from eeutils.togeotiff import toGeotiff

In [2]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

In [3]:
# import ee
# ee.Authenticate()
# ee.Initialize()

# Set Up the Working Environment
>Define the input directory containing the data

In [4]:
INPUT_DIR = 'F:\\Global'                                          # Input directory
OUTPUT_DIR = 'F:\\Global'                                         # Output directory. Update for each sensor
START = datetime(2003, 1, 31)                                     # Upload start date
END = datetime(2003, 2, 2)                                        # Upload end date
COMP = 'day'                                                      # File composite period
VAR = 'chlor_a'                                                   # Variable
BUCKET = 'gs://abcd'
DTYPE = 'l3m'                                                     # Data type, mapped or binned 
PROJ_ID = 'eqc'                                                   # map_projection = 'Equidistant Cylindrical'
SRC_SRS = 'EPSG:4087'                                             # Equidistant Cylindrical SRS of input file
TRG_SRS = 'EPSG:4326'                                             # SRS of output file | Google
FMT = '%Y-%m-%dT%H:%M:%S.%fZ'                                     # Ocean colour date format
SENSORS = {'modis': 'MODIS-Aqua', 'viirs': 'VIIRS-SNPP', 
           'seawifs': 'SeaWiFS', 'meris': 'MERIS'}                # Sensors
FILELIST = {}

# Import/Download/Read Data
> ## Global gridded L3m file
> ### Geotiff, gdal translate, upload

In [5]:
for key, val in SENSORS.items():
    if key != 'modis':
        continue
        
    init = 'A' if key == 'modis' else sen[0]
    nc = f'{INPUT_DIR}\\{val}\\{COMP.capitalize()}\\nc'
    if not os.path.isdir(nc):
        os.makedirs(nc)
    tif = f'{INPUT_DIR}\\{val}\\{COMP.capitalize()}\\tif'
    if not os.path.isdir(tif):
        os.makedirs(tif)
    
    files = glob(f'{nc}\\*.nc')
    n = len(files)
    if n < (END.toordinal() - START.toordinal()):
        FILELIST[key] = getfile(var=VAR, sen=key, dtype=DTYPE, comp=COMP, 
                                sdate=START, edate=END, output_dir=nc)
    else:
        files = []
        for d in range(START.toordinal(), END.toordinal() + 1):
            f = glob(f'{nc}/{init}{datetime.fromordinal(d).strftime("%Y%j")}*.nc')
            if len(f) == 0:
                f = getfile(var=VAR, sen=key, dtype=DTYPE, comp=COMP, output_dir=nc, 
                        sdate=datetime.fromordinal(d), edate=datetime.fromordinal(d))
                files.append(f)
            else:
                files.extend(f)
        FILELIST[key] = files

In [ ]:
def rmv_file(file: str):
    if os.name == 'nt':
        os.system(f'del /f {file} >nul')
    if os.name == 'posix':
        os.system(f'rm -f {file} 2> /dev/null')

> #### Upload manifest builder

In [6]:
def build_manifest(asset_dict:dict, disp:bool=False):

    # The enclosing object for the asset.
    asset = {
        'name': asset_dict['asset_id'],
        'tilesets': [
            {'sources': [
                {"uris": [asset_dict['source']]}]}],
        'bands': [
            {'id': asset_dict['var'],
             'missing_data': {'values': [int(asset_dict['missing_data'])]}}],
        "start_time": {"seconds": asset_dict['start_time']},
        "end_time": {"seconds": asset_dict['end_time']},
        "properties": asset_dict['attributes']
    }
    if disp:
        print(json.dumps(asset, indent=2))
    
    jason_manifest = 'temp_manifest.json'
    with open(jason_manifest, 'w') as f:
        json.dump(asset, f, indent=2)
    return jason_manifest

> #### Gdal translate function

In [7]:
def gdal_trans(infile:str, outfile:str, fill_value):
    cmd = f'gdal_translate -a_nodata {fill_value} -a_srs {TRG_SRS} {infile} {outfile}'
    exit = os.system(cmd)
    rmv_file(file=infile)
    return exit

> #### gsutil file uploader function

In [8]:
def gsupload(trg:str, bucket:str):
    basename = os.path.basename(trg)
    gc_file = f'{bucket}/{basename}'
    
    cloud_file = !gsutil ls {gc_file}
    exit = None
    if 'CommandException' in cloud_file[0]:
        print(f'\tGSUTIL\n\t{"="*10}\n\tgsutil cp {trg} {gc_file}\n')
        gs_cmd = f'gsutil cp {trg} {gc_file}'
        # To see the upload progress, use sys
        exit = os.system(gs_cmd)
    return gc_file, exit

> ### Create asset

In [9]:
def create_asset(asset_id:str):
    
    tmp_file = 'create_asset_tmp.txt'
    !earthengine --no-use_cloud_api ls -m 1 {asset_id} > {tmp_file}
    with open(tmp_file, 'r') as txt:
        res = ''.join([line.strip('\n') for line in txt.readlines()])
    
    if 'not found.' in res:
        print(f'CreateAsset\nearthengine --no-use_cloud_api create collection -p {asset_id}')
        !earthengine --no-use_cloud_api create collection -p {asset_id}
    rmv_file(file=tmp_file)

> ### List of asset files

In [ ]:
def asset_list(asset_id: str):
    tmp_file = 'ee_asset_filelist.txt'
    !earthengine --no-use_cloud_api ls {asset_id} > {tmp_file}
    with open(tmp_file, 'r') as txt:
        res = [f"{os.path.basename(line.strip('\n'))}.tif" 
               for line in txt.readlines()]
    return res

> ### Data Reader

In [10]:
def get_data(file: str, tifile: str, append):
    
    data, extend, fill_value = None, None, None
    
    with Dataset(src, 'r') as nc:
        if not os.path.isfile(tifile):
            data = nc[VAR][:]

            x0 = nc.geospatial_lon_min
            y0 = nc.geospatial_lat_min
            x1 = nc.geospatial_lon_max
            y1 = nc.geospatial_lat_max
            extent = [x0, y0, x1, y1]

        # var attributes
        fill_value = nc[VAR].getncattr('_FillValue')
        var_attrs = {key:nc[VAR].getncattr(key)
                     for key in nc[VAR].ncattrs() 
                     if not (key.startswith('_') or 
                             ('valid_m' in key) or 
                             ('display_m' in key))}
        append((datetime.strptime(nc.getncattr('time_coverage_start'), FMT), 
                datetime.strptime(nc.getncattr('time_coverage_end'), FMT), var_attrs))
                
        # append((timegm(time.strptime(nc.getncattr('time_coverage_start'), FMT)), 
        #         timegm(time.strptime(nc.getncattr('time_coverage_end'), FMT)), var_attrs))
    return data, extend, fill_value

In [11]:
def gee_upload(ee_dict: dict):
    
    fmt = '%Y-%m-%dT%H:%M:%S' 
    bsn = os.path.basename(ee_dict["source"]).strip('.tif')
    ee_cmd = f'earthengine --no-use_cloud_api upload image {ee_dict["source"]} ' \
             f'--time_start "{ee_dict["start_time"].strftime(fmt)}" ' \
             f'--time_end "{ee_dict["end_time"].strftime(fmt)}" ' \
             f'--bands {ee_dict["var"]} ' \
             f'--nodata_value {ee_dict["missing_data"]} ' \
    
    ee_cmd = ''.join([ee_cmd] + 
                     [f'--property {key}="{val}" ' 
                      for key, val in ee_dict["attributes"].items()] + 
                     [f'--asset_id="{ee_dict["asset_id"]}/{bsn}" >> gee_uploads.txt'])
    
    # !earthengine --no-use_cloud_api upload image --manifest {manifest}
    print(f'EE\n\t{"="*10}\n\t{ee_cmd}\n')
    exit = os.system(ee_cmd)
    return exit

In [12]:
# tasks = f'tasks_{datetime.today().strftime("%Y%m%d")}.txt'
# os.system(f'earthengine --no-use_cloud_api task list > {tasks}')
# with open(tasks, 'r') as txt:
#     for line in txt.readlines():
#         if 'COMPLETED' in line:
#             continue
#         if 'FAILED' in line:
#             task_id = line.split(' ')[0]
#             os.system(f'earthengine --no-use_cloud_api task cancel {task_id}')

> ### Main loop

In [7]:
for key, val in SENSORS.items():
    print(f'Sen: {val}')
    if key not in FILELIST.keys():
        continue
    # OCEANCOLOUR/MODIS-Aqua/L3M
    sen = f'{val}/L3M'
    asset_id = f'users/username/OCEANCOLOUR/{sen}'
    create_asset(asset_id=asset_id)
    
    asst_list = asset_list(asset_id=asset_id)

    nc = f'{INPUT_DIR}\\{val}\\{COMP.capitalize()}\\nc'
    tif = f'{INPUT_DIR}\\{val}\\{COMP.capitalize()}\\tif'
    nctimes = []
    gtfiles = []
    append = nctimes.append
    fill_value = None
    
    for i, src in enumerate(FILELIST[key]):
        print(f'\t{i:3}# | File: {src}')
        
        # Generate the Gtiff file
        trg = f"{tif}\\{os.path.basename(src).replace('.nc', '.tif')}"
        trg_new = trg.split('.')[0] + '.tif'
        
        data, extent, fill_value = get_data(file=src, tifile=trg_new, append=append)
        if data is not None:
            gt, srs = toGeotiff(data=data, proj_id=PROJ_ID, extent=extent, 
                                output_file=trg, fill_value=fill_value)

            # Gdal translate
            exit = gdal_trans(infile=trg, outfile=trg_new, fill_value=fill_value)
        gtfiles.append(trg_new)
        
    # Upload to GC
    for i, trg in enumerate(gtfiles):
        print(f'{i:3}# | File: {trg}')
        uirs, exit = gsupload(trg=trg)
        
        if os.path.basename(uirs) in asset_list:
            !gsutil rm {uirs}
            continue
            
        # Jason dict 
        asset_dict = {'asset_id':asset_id, 'missing_data': fill_value, 'var': VAR, 
                      'start_time': nctimes[i][0], 'end_time': nctimes[i][1], 
                      'attributes': nctimes[i][2], 'source': uirs}
        # Build upload manifest
        #manifest = build_manifest(asset_dict=asset_dict)

        # Do the upload. 
        gee_upload(ee_dict=asset_dict, bucket=BUCKET)
        

In [14]:
# cloud_file = !gsutil ls {gc_file}
# 'CommandException' in cloud_file[0]

In [6]:
# asst_id = 'users/
# tmp_file = 'eetmp.txt'
# !earthengine --no-use_cloud_api ls {asst_id} > {tmp_file}

In [8]:
# import ee
# ee.Initialize()

In [6]:
# asst_id = 'users/
# image = ee.Image(asst_id)

# import matplotlib.pyplot as plt

# fig, ax = plt.subplots()
# ax.imshow(image.load)
# image.getInfo()